### Info:
* Not all available images are used (because of storage limitation), so the the dataframes containing the information about the images are adapted
* txt-file for simple parsing is produced with each line containing: filepath,x1,y1,x2,y2,class_name

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/MLforphysicist'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# HDF5 file path
store_path = "{}/image_dataset/frsign_v1.0.h5".format(root_path)
new_store_path = "{}/image_dataset/reduced_data.h5".format(root_path)

# Read dataframes stored in HDF5 file
store = pd.HDFStore(store_path, mode='r')
new_store = pd.HDFStore(new_store_path, mode='w')
df = store.select('dataframe')
im_df = store.select('images')

In [ ]:
# list containing all video folders uploaded to gdrive
folder_list = os.listdir("{}/image_dataset/images".format(root_path))
#print(folder_list)

# extract the folder which contains the image
splitted_path = im_df["fullpath"]
splitted_path = splitted_path.str.split("/", expand=True)
#print(splitted_path)
#print(im_df["fullpath"].iloc[0])

new_im_df = pd.DataFrame()

# appending images to new dataframe, that are in the uploaded folders
for i in folder_list:
  new_im_df = new_im_df.append(im_df[splitted_path[0] == i])

print(im_df)
print(new_im_df) 

                                                         fullpath    x  ...   w    h
sequence image                                                          ...         
83       0      RecFile_1_20181011_153137_pointgrey_flycapture...  882  ...  15   21
         1      RecFile_1_20181011_153137_pointgrey_flycapture...  882  ...  15   21
         2      RecFile_1_20181011_153137_pointgrey_flycapture...  882  ...  15   21
         3      RecFile_1_20181011_153137_pointgrey_flycapture...  882  ...  15   21
         4      RecFile_1_20181011_153137_pointgrey_flycapture...  882  ...  15   21
...                                                           ...  ...  ...  ..  ...
1149     677    RecFile_1_20190124_135313_pointgrey_flycapture...  583  ...  75  177
         678    RecFile_1_20190124_135313_pointgrey_flycapture...  583  ...  75  177
         679    RecFile_1_20190124_135313_pointgrey_flycapture...  583  ...  75  177
         680    RecFile_1_20190124_135313_pointgrey_flycapture...

In [ ]:
#appending string ".avi" to match dataframe
folder_list = [x+".avi" for x in folder_list]
#print(folder_list)

new_df = pd.DataFrame()

# appending video sequences to new dataframe, that are in the uploaded folders
for i in folder_list:
  new_df = new_df.append(df[df["video_name"] == i])

print(df)
print(new_df)

         CameraInfo_bayerTileFormat  ... image_format
sequence                             ...             
83                             RGGB  ...         PNG8
124                            RGGB  ...         PNG8
128                            RGGB  ...         PNG8
129                            RGGB  ...         PNG8
164                            RGGB  ...         PNG8
...                             ...  ...          ...
3829                           RGGB  ...         PNG8
4132                           RGGB  ...         PNG8
3887                           GBRG  ...         PNG8
2208                           GBRG  ...         PNG8
1149                           RGGB  ...         PNG8

[393 rows x 14 columns]
         CameraInfo_bayerTileFormat  ... image_format
sequence                             ...             
380                            RGGB  ...         PNG8
382                            RGGB  ...         PNG8
352                            RGGB  ...         PNG8
355

In [ ]:
# replace target (type) with simpler string
print(new_df["type"].unique())
new_df["type"] = new_df["type"].replace("Chassis A (3 feux verticaux)", "A")
new_df["type"] = new_df["type"].replace("Chassis C (5 feux verticaux)", "C")
new_df["type"] = new_df["type"].replace("Chassis H (6 & 3 feux en S)", "H")
new_df["type"] = new_df["type"].replace("Chassis F (6 & 1 feux en L inverse)", "F")
new_df["type"] = new_df["type"].replace("Chassis R (6 feux dans un disque)", "R")
new_df["type"] = new_df["type"].replace("Chassis ID3 (3 feux horizontaux)", "ID3")
new_df["type"] = new_df["type"].replace("Chassis ID2 (2 feux horizontaux)", "ID2")
print(new_df["type"].unique())

['Chassis A (3 feux verticaux)' 'Chassis ID3 (3 feux horizontaux)'
 'Chassis H (6 & 3 feux en S)' 'Chassis C (5 feux verticaux)'
 'Chassis F (6 & 1 feux en L inverse)' 'Chassis ID2 (2 feux horizontaux)'
 'Chassis R (6 feux dans un disque)']
['A' 'ID3' 'H' 'C' 'F' 'ID2' 'R']


In [ ]:
# create a list containing the target variable for each image
sequence = new_im_df.index.get_level_values("sequence")
target = []

for i in sequence:
  target.append(new_df["type"][i])

new_im_df["target"] = target

# simple parsing needs x1 x2 and y1 y2
new_im_df["w"] = new_im_df["w"] + new_im_df["x"]
new_im_df["h"] = new_im_df["h"] + new_im_df["y"]

# full filepath
list_rootpath = [root_path+"/image_dataset/images/"]*len(new_im_df)
print(list_rootpath)
print(len(list_rootpath))
new_im_df["fullpath"] = list_rootpath+new_im_df["fullpath"]

In [ ]:
new_im_df

fullpath  ...  target
sequence image                                                     ...        
380      0      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         1      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         2      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         3      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         4      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
...                                                           ...  ...     ...
2643     208    gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         209    gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         210    gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         211    gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         212    gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A

[50379 rows x 6 columns]

In [ ]:
# create txt file for simple parsing (in the moment just for one folder)
print(new_df["video_name"].iloc[0])
print(new_df[new_df["video_name"] == new_df["video_name"].iloc[0]])

selected_sequences = new_df[new_df["video_name"] == new_df["video_name"].iloc[0]].index
new_im_df = new_im_df.loc[selected_sequences]
print(new_im_df)

new_im_df.to_csv("{}/firsttrain.txt".format(root_path), header=False, index=False,)
#np.savetxt("{}/firsttrain.txt".format(root_path), new_im_df.values, delimiter=",")

RecFile_1_20181115_092834_pointgrey_flycapture2_1_ipl_image.avi
         CameraInfo_bayerTileFormat  ... image_format
sequence                             ...             
380                            RGGB  ...         PNG8
382                            RGGB  ...         PNG8
352                            RGGB  ...         PNG8
355                            RGGB  ...         PNG8
391                            RGGB  ...         PNG8
350                            RGGB  ...         PNG8
386                            RGGB  ...         PNG8
390                            RGGB  ...         PNG8

[8 rows x 14 columns]
                                                         fullpath  ...  target
sequence image                                                     ...        
380      0      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         1      gdrive/My Drive/MLforphysicist/data/images/Rec...  ...       A
         2      gdrive/My Drive/MLforphysicist/data/image

In [ ]:
new_store.put('dataframe', new_df, format='table', data_columns=True)
new_store.put('images', new_im_df, format='table', data_columns=True)

In [ ]:
# Close store
new_store.close()
store.close()